# Client 2 Object Detection

In [1]:
# 최적의 모델을 얻고 싶을 경우, 세션 재시작 후 CPU 코어 8개로 설정.
import os
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

In [2]:
!pip install -r ./requirements.txt.client

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
!pip install pybboxes albumentations==1.3.0 --upgrade typing-extensions torch

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [4]:
import albumentations as A
from augmentation import get_bboxes_list, apply_aug
import yaml

In [5]:
import shutil
import zipfile
import numpy as np
import pandas as pd
from PIL import Image
import PIL
import json
import sys
import shutil
from distutils.dir_util import copy_tree
# import imagesize
# import ptitprince as pt
from shutil import copyfile
import time
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.patches as mpatches
import seaborn as sns
import re
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
import random
import torch
from tqdm.notebook import tqdm
import cv2
from collections import defaultdict
import glob

In [6]:
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True)
    os.environ["PYTHONHASHSEED"] = str(seed)
    os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"
seed_everything()

In [7]:
client = 'C2_PCB' # if client 2
class_name = ['SCRATCH', 'DELAMINATION', 'POP_CORN_1', 'POP_CORN_2'] # BOXING_TXT_1013

## 데이터 증강

In [8]:
# # 데이터 증강했으면 재증강하지 않는다.
# # YAML 데이터 생성
# data = {
#     "inp_img_pth": f"./{client}/train/images/",
#     "inp_lab_pth": f"./{client}/train/labels/",
#     "out_img_pth": f"./{client}/train/images/",
#     "out_lab_pth": f"./{client}/train/labels/",
#     "transformed_file_name": "AUG",
#     "CLASSES": class_name
# }

# # YAML 파일에 데이터 쓰기
# with open(f"./{client}/train_contants.yaml", 'w') as yaml_file:
#     yaml.dump(data, yaml_file)

# with open(f"./{client}/train_contants.yaml", 'r') as stream:
#     CONSTANTS = yaml.safe_load(stream)

# imgs = os.listdir(CONSTANTS["inp_img_pth"])
# for img_file in tqdm(imgs):
#     file_name = img_file[:-4]
#     if 'ipynb_checkpo' in file_name:
#         continue
#     image = cv2.imread(os.path.join(CONSTANTS["inp_img_pth"], img_file))
#     lab_pth = os.path.join(CONSTANTS["inp_lab_pth"], file_name + '.txt')
#     album_bboxes = get_bboxes_list(lab_pth, CONSTANTS['CLASSES'])
#     print(file_name)
#     # x번의 데이터 증강을 수행하고 각각의 결과를 저장 range(1, x) <- 여기를 변경
#     for i in range(1, 5):
#         aug_file_name = f"{file_name}_AUG_{i}"
#         try:
#             apply_aug(image, album_bboxes, CONSTANTS["out_lab_pth"], CONSTANTS["out_img_pth"], aug_file_name, CONSTANTS['CLASSES'])
#         except Exception as e:
#             print(f"An error occurred for {aug_file_name}: {str(e)}")
#             continue  # 에러 발생 시 해당 부분을 건너뛰고 계속 진행


## YAML 파일 생성

In [9]:
# yaml 파일 생성
data = { 'train' : os.getcwd() + f'/{client}/train/images',
         'val' : os.getcwd() + f'/{client}/valid/images/',
         'test' : os.getcwd() + f'/{client}/test/images',
         'names' : class_name,
         'nc' : len(class_name) }

with open(f'./{client}/PCB.yaml', 'w') as f:
    yaml.dump(data, f)

with open(f'./{client}/PCB.yaml', 'r') as f:
    pcb_yaml = yaml.safe_load(f)
    display(pcb_yaml)

{'names': ['SCRATCH', 'DELAMINATION', 'POP_CORN_1', 'POP_CORN_2'],
 'nc': 4,
 'test': '/home/work/KISTI_PCB2/C2_PCB/test/images',
 'train': '/home/work/KISTI_PCB2/C2_PCB/train/images',
 'val': '/home/work/KISTI_PCB2/C2_PCB/valid/images/'}

In [10]:
!cat ./{client}/PCB.yaml

names:
- SCRATCH
- DELAMINATION
- POP_CORN_1
- POP_CORN_2
nc: 4
test: /home/work/KISTI_PCB2/C2_PCB/test/images
train: /home/work/KISTI_PCB2/C2_PCB/train/images
val: /home/work/KISTI_PCB2/C2_PCB/valid/images/


## Yolov5 Train

In [11]:
if os.getcwd() != '/home/work/KISTI_PCB2/yolov5':
    os.chdir('./yolov5')
os.getcwd()

'/home/work/KISTI_PCB2/yolov5'

In [12]:
!python train.py --data "/home/work/KISTI_PCB2/{client}/PCB.yaml" \
    --epochs 300 \
    --patience 15 \
    --batch 32 \
    --imgsz 640 \
    --weights 'yolov5n.pt' \
    --name '{client}_results' \
    --exist-ok

train: weights=yolov5n.pt, cfg=, data=/home/work/KISTI_PCB2/C2_PCB/PCB.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=C2_PCB_results, exist_ok=True, quad=False, cos_lr=False, label_smoothing=0.0, patience=15, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 3 commits. Use 'git pull' or 'git clone https://github.com/ultralytics/yolov5' to update.
YOLOv5 🚀 v7.0-224-g6262c7f Python-3.8.10 torch-2.1.0+cu121 CUDA:0 (Quadro RTX 8000, 48593MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls

## Test Yolo(Client 2 Centralized)

In [ ]:
!python val.py --data "/home/work/KISTI_PCB2/{client}/PCB.yaml" --weights "/home/work/KISTI_PCB2/yolov5/runs/train/{client}_train_results/weights/best.pt" --task 'test' --name '{client}_result_final(central)' --exist-ok

# Do Federated Learning (Client 2)

In [13]:
from fedClient import *

torch.hub.load('ultralytics/yolov5', 'yolov5n')
token = 'tGD6z2Yw7yqiY1g6'

Using cache found in /home/work/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-10-19 Python-3.8.10 torch-2.1.0+cu121 CUDA:0 (Quadro RTX 8000, 48593MiB)

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients, 4.5 GFLOPs
Adding AutoShape... 


In [14]:
num_round = 15
for n in range(1, num_round+1):
        print(f'Round {n} Start \n')

        # n-1 round 모델 기반 Client 학습 모델 불러오기
        with open(f"/home/work/KISTI_PCB2/yolov5/runs/train/{client}_results/weights/best.pt", "rb") as f:
            model_bytes = f.read()
        
        # n round에서의 연합 학습
        federate('http://fl-aggregator.kro.kr', token = token, model_bytes = str(model_bytes), round = n)
         
        # n round client 학습
        if n != num_round:
            !python train.py --data "/home/work/KISTI_PCB2/{client}/PCB.yaml" \
                --epochs 300 \
                --patience 15 \
                --batch 32 \
                --imgsz 640 \
                --weights "model{n}_{token}.pt" \
                --name '{client}_results' \
                --exist-ok

Round 1 Start 

Connected to Flask-SocketIO server


Using cache found in /home/work/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-10-19 Python-3.8.10 torch-2.1.0+cu121 CUDA:0 (Quadro RTX 8000, 48593MiB)

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients, 4.5 GFLOPs
Adding AutoShape... 


train: weights=model1_tGD6z2Yw7yqiY1g6.pt, cfg=, data=/home/work/KISTI_PCB2/C2_PCB/PCB.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=C2_PCB_results, exist_ok=True, quad=False, cos_lr=False, label_smoothing=0.0, patience=15, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 3 commits. Use 'git pull' or 'git clone https://github.com/ultralytics/yolov5' to update.
YOLOv5 🚀 v7.0-224-g6262c7f Python-3.8.10 torch-2.1.0+cu121 CUDA:0 (Quadro RTX 8000, 48593MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.

Using cache found in /home/work/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-10-19 Python-3.8.10 torch-2.1.0+cu121 CUDA:0 (Quadro RTX 8000, 48593MiB)

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients, 4.5 GFLOPs
Adding AutoShape... 


train: weights=model2_tGD6z2Yw7yqiY1g6.pt, cfg=, data=/home/work/KISTI_PCB2/C2_PCB/PCB.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=C2_PCB_results, exist_ok=True, quad=False, cos_lr=False, label_smoothing=0.0, patience=15, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 3 commits. Use 'git pull' or 'git clone https://github.com/ultralytics/yolov5' to update.
YOLOv5 🚀 v7.0-224-g6262c7f Python-3.8.10 torch-2.1.0+cu121 CUDA:0 (Quadro RTX 8000, 48593MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.

...

Using cache found in /home/work/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-10-19 Python-3.8.10 torch-2.1.0+cu121 CUDA:0 (Quadro RTX 8000, 48593MiB)

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients, 4.5 GFLOPs
Adding AutoShape... 


train: weights=model13_tGD6z2Yw7yqiY1g6.pt, cfg=, data=/home/work/KISTI_PCB2/C2_PCB/PCB.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=C2_PCB_results, exist_ok=True, quad=False, cos_lr=False, label_smoothing=0.0, patience=15, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 3 commits. Use 'git pull' or 'git clone https://github.com/ultralytics/yolov5' to update.
YOLOv5 🚀 v7.0-224-g6262c7f Python-3.8.10 torch-2.1.0+cu121 CUDA:0 (Quadro RTX 8000, 48593MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0

Using cache found in /home/work/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-10-19 Python-3.8.10 torch-2.1.0+cu121 CUDA:0 (Quadro RTX 8000, 48593MiB)

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients, 4.5 GFLOPs
Adding AutoShape... 


train: weights=model14_tGD6z2Yw7yqiY1g6.pt, cfg=, data=/home/work/KISTI_PCB2/C2_PCB/PCB.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=C2_PCB_results, exist_ok=True, quad=False, cos_lr=False, label_smoothing=0.0, patience=15, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 3 commits. Use 'git pull' or 'git clone https://github.com/ultralytics/yolov5' to update.
YOLOv5 🚀 v7.0-224-g6262c7f Python-3.8.10 torch-2.1.0+cu121 CUDA:0 (Quadro RTX 8000, 48593MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0

Using cache found in /home/work/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-10-19 Python-3.8.10 torch-2.1.0+cu121 CUDA:0 (Quadro RTX 8000, 48593MiB)

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients, 4.5 GFLOPs
Adding AutoShape... 


## Test Federated Learning Model (inference)

In [ ]:
if os.getcwd() != '/home/work/KISTI_PCB2/yolov5':
    os.chdir('./yolov5')
os.getcwd()

In [ ]:
for n in range(1, n):
    !python val.py --data "/home/work/KISTI_PCB2/{client}/PCB.yaml" --weights "model{n}_{token}.pt" --task 'test' --name 'PCB_result_final(federated)_round{n}' --exist-ok

## Detect Federated Learning Model

In [ ]:
!python detect.py --source "/home/work/KISTI_PCB2/{client}/test/images"\
    --weights "model{num_round}_{token}.pt" \
    --exist-ok \
    --line-thickness 2 \
    --name 'PCB_detect_results(federated)'